In [37]:
!pip install dash_bootstrap_components

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.2/221.2 kB 2.6 MB/s eta 0:00:00a 0:00:01


In [15]:
import datetime
from dash import Dash, dcc, html, Input, Output, callback

app = Dash(__name__)


datepickerrange = dcc.DatePickerRange(
        id='datepickerrange',
        display_format='DD/MM/YYYY',
        min_date_allowed=datetime.date(2021, 1, 1),
        max_date_allowed=datetime.date.today(),
        initial_visible_month=datetime.date.today(),
        end_date=datetime.date.today()
    )





app.layout = html.Div([
    dcc.DatePickerRange(
        id='my-date-picker-range',
        display_format='DD/MM/YYYY',
        min_date_allowed=datetime.date(2021, 1, 1),
        max_date_allowed=datetime.date.today(),
        initial_visible_month=datetime.date.today(),
        end_date=datetime.date.today()
    ),
    html.Div(id='output-container-date-picker-range')
])


@callback(
    Output('output-container-date-picker-range', 'children'),
    Input('my-date-picker-range', 'start_date'),
    Input('my-date-picker-range', 'end_date'))
def update_output(start_date: "None or str", end_date: "None or str"):

    print(start_date, type(start_date), end_date, type(end_date))
    
    string_prefix = 'You have selected: '
    if start_date is not None:
        start_date_object = datetime.date.fromisoformat(start_date)
        start_date_string = start_date_object.strftime('%B %d, %Y')
        string_prefix = string_prefix + 'Start Date: ' + start_date_string + ' | '
    if end_date is not None:
        end_date_object = datetime.date.fromisoformat(end_date)
        end_date_string = end_date_object.strftime('%B %d, %Y')
        string_prefix = string_prefix + 'End Date: ' + end_date_string
    if len(string_prefix) == len('You have selected: '):
        return 'Select a date to see it displayed here'
    else:
        return string_prefix


if __name__ == '__main__':
    app.run(debug=True)


None <class 'NoneType'> 2023-11-10 <class 'str'>


In [44]:
"""
BesserEsser Plotly Dashboard.
Description and explanations for the code will be placed her

TODO: README.md
- callback must be on this module
- ...
"""


# TODO: edit the imports
import time  # for demo/debug purposes - delete?
import datetime
import numpy as np, pandas as pd
import plotly.express as px  #???
from dash import Dash, dcc, html, Input, Output, State, callback, ctx, no_update
from dash.dash_table import DataTable
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
from io import StringIO
from PIL import Image
from utilities import get_data_from_database
# from units import unit_0, ...
# from utilities import get_default_values


# Debugging mode?
DEBUG = True


# Define the style and colors (optional)
px.defaults.color_discrete_sequence = px.colors.qualitative.T10
px.defaults.color_continuous_scale = px.colors.sequential.Jet

css = {
        "color": "black",
        "backgroundColor": "white",
        "fontSize": 12,
        "fontFamily":"Arial",
        "text-align": "left"
            }



def instantiate_debug_table(id=None):
    PAGE_SIZE = 20
    return html.Div(style={'display': 'block' if DEBUG else 'none'},
                    children=[html.H4("data table for debugging:"),
                              DataTable(page_size=PAGE_SIZE, id=id) if id else DataTable(page_size=PAGE_SIZE)])



# Define your units here

BANNER_PATH = "assets/banner.png"
header = html.Div(
    style=css,  # just to show how to use css here
    children=[html.Img(src=Image.open(BANNER_PATH), height=150, width="100%")]
)

unit_0 = html.Div([
        html.Div([
            dcc.Input(placeholder="Account ID", id="inputbox", type='number', debounce=True),
            html.Button("Go", id='button', n_clicks=0),
            html.P(id='warning_message', style={'color': 'red'})
                ]),
        html.H2("", id="displaying_data_message", style={'textAlign':'center'}),
        dcc.Store(id="store"),
    ])

#style={'visibility': 'visible'}

datepickerrange = dcc.DatePickerRange(
        display_format='DD/MM/YYYY',
        min_date_allowed=datetime.date(2021, 1, 1),
        max_date_allowed=datetime.date.today(),
        initial_visible_month=datetime.date.today(),
        end_date=datetime.date.today(),
        id='unit_1_component_1')

unit_1 = html.Div([
        html.H2("Graph 1"),
        datepickerrange,
        dcc.Dropdown(options=["last month", "last week"], value="last month", id="unit_1_component_2"),
        #dcc.RadioItems(options=["a", "b", "c"], value='a', id="unit_1_component_3"),
        dcc.Graph(id="unit_1_graph_1"),
        instantiate_debug_table(id="unit_1_table_1")
    ])

unit_2 = html.Div([
        html.H2("Graph 2"),
        dcc.Dropdown(options=[{'label':"Frühstück", 'value':"BREAKFAST"}, 
                              {'label':"Mittagessen", 'value':"LUNCH"}, 
                              {'label':"Abendessen", 'value':"DINNER"}], 
                     value="BREAKFAST", id="unit_2_component_1"),
        dcc.RadioItems(options=["x", "y", "z"], value='x', id="unit_2_component_2"),
        dcc.Graph(id="unit_2_graph_1"),
        instantiate_debug_table(id="unit_2_table_1")
    ])

unit_3 = html.Div([
        html.H2("Graph 3"),
        dcc.RadioItems(options=["I", "II", "III"], value='I', id="unit_3_component_1"),
        dcc.Dropdown(options=["F", "M"], value="F", id="unit_3_component_2"),
        dcc.Graph(id="unit_3_graph_1"),
        instantiate_debug_table(id="unit_3_table_1")
    ])


# Instantiate an application object
app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])
app = Dash(__name__)

# Define the layout
app.layout = html.Div([header, unit_0, unit_1, unit_2, unit_3], style=css)





##### UTILITY FUNCTIONS FOR THE CALLBACKS#####

SELECTOR_CLASSES = (dcc.Dropdown, dcc.RadioItems)  # extend this list if necessary


def get_default_values(unit):
    """
    TODO: docs
    """
    SELECTOR_CLASSES = (dcc.Dropdown, dcc.RadioItems)  # extend this list if necessary

    collector = []

    for e in unit.children:
        if e.__class__ in SELECTOR_CLASSES and hasattr(e, 'options'):
            if type(e.options[0]) is dict:
                collector.append(e.options[0]['value'])
            else:
                collector.append(e.options[0])
        elif True:
            "add more hard-coded logic as necessary, e.g. for start_date/end_date in a date-picker"
    
    return collector


def get_dash_components(seq: "array or dash.html.Div"):
    """
    Get dash components from an array or dash.html.Div recusively
    """
    # Update this arrray as necessary
    DASH_COMPONENTS_CLASSES = (dcc.Dropdown, dcc.RadioItems, dcc.Slider, 
                               dcc.DatePickerRange, dcc.DatePickerSingle, dcc.Graph, DataTable)
    collector = []

    def recurse(element):
        # base case
        if type(element) in DASH_COMPONENTS_CLASSES:
            collector.append(element)
            return
        elif type(element) is str or not(hasattr(element, '__len__')) or not(hasattr(element, 'children') and element.children):
            return
        
        # recursive case
        for e in element.children if hasattr(element, 'children') else element:
            recurse(e)
            
    recurse(seq)
    return collector
    


def get_callback_args(unit, parent_unit=None):
    """
    A helper function to make the decorating of a callback function easy.
    Returns a tuple of Output/Input objects.
    This tuple is to be unpacked when defining the decoration, like so: *get_callback_args(...)
    unit is a Div container with dcc components
    representing an individual plot
    """

    collector = [[], [Input(component_id='store', component_property='data')], [], []]
    
    for e in get_dash_components(unit):
        if e.__class__ is dcc.DatePickerRange:
            collector[0].append(Output(e, component_property='start_date'))
            collector[0].append(Output(e, component_property='end_date'))
            collector[-1].append(Input(e, component_property='start_date'))
            collector[-1].append(Input(e, component_property='end_date'))
        if hasattr(e, 'value'):
            collector[0].append(Output(e, component_property='value'))
            collector[-1].append(Input(e, component_property='value'))
        elif e.__class__ is dcc.Graph:
            collector[0].append(Output(e, component_property='figure'))
        elif e.__class__ is DataTable:
            collector[0].append(Output(e, component_property='data'))
        elif True:
            "add more hard-coded logic as necessary, e.g. for start_date/end_date in a date-picker"
            
    if parent_unit:
        for e in get_dash_components(parent_unit):
            if e.__class__ is dcc.DatePickerRange:
                collector[2].append(Input(e, component_property='start_date'))
                collector[2].append(Input(e, component_property='end_date'))
            elif hasattr(e, 'value'):
                collector[2].append(Input(e, component_property='value'))
            elif True:
                "add more hard-coded logic as necessary, e.g. for start_date/end_date in a date-picker"
    
    return tuple(sum(collector, []))



"""
@callback(
    Output(component_id='unit_1_component_1', component_property='value'),
    Output(component_id='unit_1_component_2', component_property='value'),
    Output(component_id='unit_1_graph_1', component_property='figure'),
    Output(component_id='unit_1_table_1', component_property='data'),
    Input(component_id='store', component_property='data'),
    Input(component_id='unit_1_component_1', component_property='value'),
    Input(component_id='unit_1_component_2', component_property='value')
)
"""


t = get_callback_args(unit_3, parent_unit=unit_1)
t

(<Output `unit_3_component_1.value`>,
 <Output `unit_3_component_2.value`>,
 <Output `unit_3_graph_1.figure`>,
 <Output `unit_3_table_1.data`>,
 <Input `store.data`>,
 <Input `unit_1_component_1.start_date`>,
 <Input `unit_1_component_1.end_date`>,
 <Input `unit_1_component_2.value`>,
 <Input `unit_3_component_1.value`>,
 <Input `unit_3_component_2.value`>)

In [31]:
unit_1

Div([H2('Graph 1'), Dropdown(options=['A', 'B'], value='A', id='unit_1_component_1'), RadioItems(options=['a', 'b', 'c'], value='a', id='unit_1_component_2'), Graph(id='unit_1_graph_1'), Div(children=[H4('data table for debugging:'), DataTable(page_size=10, id='d4713d60-c8a7-0639-eb11-67b367a9c378')], style={'display': 'block'})])

In [65]:
df['date'].min().to_pydatetime()

datetime.datetime(2022, 1, 13, 0, 0)

Timestamp('2022-01-13 00:00:00')